# MINE API Examples
MINE is a database of predicted metabolic reactions, based on known biochemistry. You can use the MINE API to:

1. Perform a database search, including MINE compound search by similarity or substructure
2. Retrieve lists of compounds, operators, and reactions
3. Perform an MS search (requires MS data)
4. Perform an MS2 search (requires MS2 data)
5. Download predicted spectra for MINE compounds
6. Get a list of possible adducts for metabolomics searches

Examples are provided below for each of these requests. The mongotest MINE database, which contains 26 compounds, 8 reactions, and 2 operators, is used for all examples.

**Note:** If using python requests library, make sure verify argument to get() or post() method is set to False. We are seeking to fix this in the future.

## Setup

In [1]:
import requests
import json
import certifi

In [2]:
BASE_URL = 'https://minedatabase.ci.northwestern.edu/mineserver/'

DB_NAME = 'kegg_lte600_500mcy'  # KEGG MINE 2.0
#DB_NAME = 'ecocyc_lte600_500mcy'  # EcoCyc MINE 2.0
#DB_NAME = 'ymdb_lte600_500mcy'  # YMDB MINE 2.0

## 1. Perform a database compound search
All searches are GET requests.

**Endpoints**:

- **Quick Search**: .../mineserver/quick-search/&lt;db_name>/q=&lt;query>

- **Similarity Search**: ...mineserver/similarity-search/&lt;db_name>/smiles=&lt;smiles>/&lt;min_tc>/&lt;limit>  
....where &lt;min_tc> and &lt;limit> are optional.

- **Substructure Search**: .../mineserver/substructure-search/&lt;db_name>/smiles=&lt;smiles>/&lt;limit>  
....where &lt;limit> is optional.

- **Model Search**: .../mineserver/model-search/q=&lt;query>

**URL Arguments**:
- &lt;db_name>: Name of MINE database
- &lt;query>: Search query - format depends on the search:
 - Quick Search: a MINE id, KEGG organism code, ModelSEED id, Inchikey, or Name
 - Model Search: KEGG organism code or organism name
 - Database Search: a valid Mongo query
- &lt;smiles>: SMILES string describing molecular structure (or substructure for substructure search)
- &lt;min_tc>: (optional) minimum Tanimoto coefficient for similarity scoring
- &lt;limit>: (optional) maximum number of results to return

Quick search, similarity search, and substructure search will likely be the most common use cases. Model search can be used to see whether MINE has compounds linked to a specific organism. Database search can be used for completely customizable searches as it just requires a valid search query using Mongo syntax.

### Example 1a - Quick Search
Here, we perform a quick search based on a compound KEGG ID (can also do InChI key or MINE ID)

In [3]:
url = BASE_URL + f'quick-search/{DB_NAME}/q=C00583'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Formula': 'C3H8O2',
  'KEGG_id': 'C00583',
  'MINE_id': 41,
  'SMILES': 'CC(O)CO',
  '_id': 'Cb1654fad58a20fa68742b1bd114fd58c9236cb34',
  'logP': -0.6405000000000001}]

### Example 1b - Similarity Search
Here, we perform a similarity search using pyruvate as the query molecule. Pyruvate is in the KEGG MINE 2.0 database so it is returned. D-allose is also returned.

We set the similarity to be above 0.75 (Tanimoto similarity using RDKit fingerprints) and want to return up to 100 results.

In [4]:
url = BASE_URL + f'similarity-search/{DB_NAME}/smiles=CC(=O)C(=O)O/0.75/100'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Charge': 0,
  'Formula': 'C3H4O3',
  'Inchi': 'InChI=1S/C3H4O3/c1-2(4)3(5)6/h1H3,(H,5,6)',
  'Inchikey': 'LCTONWCANYUPML-UHFFFAOYSA-N',
  'KEGG_id': 'C00022',
  'MINES': ['KEGG_mcy90p_lte500',
   'test',
   'kegg_lte400_500mcy',
   'test_db',
   'kegg_lte600_500mcy',
   'ATLAS_lte800_500mcy',
   'ecocyc_lte600_500mcy',
   'ymdb_lte600_500mcy',
   'adp1_met_aro2g_allfilters',
   'adp1_met_aro2g_thermofilter',
   'adp1_met_aro2g_feasfilter',
   'adp1_met_aro2g_150lt40_thermofilter',
   'adp1_met_aro2g_150lt40_allfilters',
   'adp1_met_aro2g_150lt40_stan_thermofilter',
   'adp1_metab_arom_2g',
   'ecoli_metab_100t300r200'],
  'MINE_id': 32182,
  'Mass': 88.016043988,
  'RDKit_fp': [40,
   45,
   50,
   51,
   64,
   89,
   94,
   97,
   110,
   111,
   129,
   130,
   149,
   159,
   167,
   192,
   197,
   199,
   200,
   209,
   236,
   240,
   254,
   277,
   284,
   293,
   297,
   315,
   336,
   350,
   371,
   388,
   395,
   417,
   481,
   485],
  'SMILES': 'CC(=O)C(=O)O',
  '

In [5]:
len(json.loads(response.content))

3

### Example 1c - Substructure Search
Here, we perform a substructure search using a part of the D-glucose molecule (just a C-C-C-O structure)

In [6]:
url = BASE_URL + f'substructure-search/{DB_NAME}/smiles=CCCO'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Formula': 'C3H8O2',
  'KEGG_id': 'C00583',
  'MINE_id': 41,
  'SMILES': 'CC(O)CO',
  '_id': 'Cb1654fad58a20fa68742b1bd114fd58c9236cb34',
  'logP': -0.6405000000000001},
 {'Formula': 'C4H10O2',
  'KEGG_id': None,
  'MINE_id': 78,
  'SMILES': 'COCC(C)O',
  '_id': 'Cfd5c53d28c83351c8dbedb0bd4f5aa6fadada2a3',
  'logP': 0.013600000000000112},
 {'Formula': 'C24H42N7O17P3S',
  'KEGG_id': None,
  'MINE_id': 116,
  'SMILES': 'CC(O)CSCCNC(=O)CCNC(=O)[C@H](O)C(C)(C)COP(=O)(O)OP(=O)(O)OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C@H](O)[C@@H]1OP(=O)(O)O',
  '_id': 'C49d911674d1f83fc55e7674aa46b6027197c97a2',
  'logP': -1.4810999999999948},
 {'Formula': 'C3H9O5P',
  'KEGG_id': 'C03894',
  'MINE_id': 126,
  'SMILES': 'CC(O)COP(=O)(O)O',
  '_id': 'C4b93cd8a35140522c4b77b47204e6b85805c68e4',
  'logP': -0.5235000000000001},
 {'Formula': 'C3H10O8P2',
  'KEGG_id': None,
  'MINE_id': 129,
  'SMILES': 'CC(O)COP(=O)(O)OP(=O)(O)O',
  '_id': 'C864294557602c41b182373eb2d1b669072a9823e',
  'logP': -0.4065000000000001},

In [7]:
len(json.loads(response.content))

100

### Example 1d - Model Search
Here, we see if an organism code exists in the MINE database.

In [8]:
url = BASE_URL + 'model-search/q=eco'  # E. coli
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['eco', 'ecos']

## 2. Retrieve Lists of Compounds, Reactions, and Operators
### NOTE: The compound and reaction endpoints are no longer functional in MINE 2.0 as database size increased and the request now becomes too large. Updates to handle this are currently in development.
Compounds, Reactions, and Operators are each a collection within each MINE database. get-ids is a GET request while get-comps, get-rxns, and get-ops are POST requests. get-op-w-rxns is a GET request.

**Endpoints**:
- **Get IDs**: .../mineserver/get-ids/&lt;db_name>/&lt;collection_name>
- **Get Compounds**: .../mineserver/get-comps/&lt;db_name>
- **Get Reactions**: .../mineserver/get-rxns/&lt;db_name>
- **Get Operators**: .../mineserver/get-ops/&lt;db_name>
- **Get Operator with Reactions**: .../mineserver/get-op-w-rxns/&lt;db_name>/&lt;op_id>

**URL Arguments**:
- &lt;db_name>: name of MINE database
- &lt;collection_name>: name of collection with in MINE database
- &lt;query>: get only IDs for documents matching this query (in Mongo syntax)
- &lt;op_id>: operator ID (e.g. 1.1.-1.h) or Mongo ID for operator document

**JSON Arguments (for POST requests)**:
- id_list: List of IDs. Can be either MINE IDs or Mongo document IDs.
 - can specify for get-comps, get-rxns, or get-ops POST requests

A general workflow to retrieve all the compound information for a MINE database could be to (1) get all IDs for compounds in the compound collection (via get-ids), and to then (2) get info on all those compounds via get-comps.

### Example 2a - Get IDs
Here, we retrieve IDs of all compounds, reactions, and operators in the KEGG MINE 2.0 database. For compounds and reactions, this is the Mongo document ID. For operators, this is the operator name (e.g. 2.7.1.a).

Note that these requests can take a while. If you already know the IDs of the compounds you want, see 2b.

#### For Compounds

In [ ]:
url = BASE_URL + f'get-ids/{DB_NAME}/compounds'
response = requests.get(url, verify=False)
json.loads(response.content)

#### For Reactions

In [ ]:
url = BASE_URL + f'get-ids/{DB_NAME}/reactions'
response = requests.get(url, verify=False)
json.loads(response.content)

#### For Operators

In [10]:
url = BASE_URL + f'get-ids/{DB_NAME}/operators'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['rule0001',
 'rule0002',
 'rule0003',
 'rule0004',
 'rule0005',
 'rule0006',
 'rule0007',
 'rule0008',
 'rule0009',
 'rule0010',
 'rule0011',
 'rule0012',
 'rule0013',
 'rule0014',
 'rule0015',
 'rule0016',
 'rule0017',
 'rule0018',
 'rule0019',
 'rule0020',
 'rule0021',
 'rule0022',
 'rule0023',
 'rule0024',
 'rule0025',
 'rule0026',
 'rule0027',
 'rule0028',
 'rule0029',
 'rule0030',
 'rule0031',
 'rule0032',
 'rule0033',
 'rule0034',
 'rule0035',
 'rule0036',
 'rule0037',
 'rule0038',
 'rule0039',
 'rule0040',
 'rule0041',
 'rule0042',
 'rule0043',
 'rule0044',
 'rule0045',
 'rule0046',
 'rule0047',
 'rule0048',
 'rule0049',
 'rule0050',
 'rule0051',
 'rule0052',
 'rule0053',
 'rule0054',
 'rule0055',
 'rule0056',
 'rule0057',
 'rule0058',
 'rule0059',
 'rule0060',
 'rule0061',
 'rule0062',
 'rule0063',
 'rule0064',
 'rule0065',
 'rule0066',
 'rule0067',
 'rule0068',
 'rule0069',
 'rule0070',
 'rule0071',
 'rule0072',
 'rule0073',
 'rule0074',
 'rule0075',
 'rule0076',
 'rule0077',

In [11]:
len(json.loads(response.content))

500

### Example 2b - Get Compounds
Here, we get info for three compounds returned from get-ids for the compound collection. The return_extra_info flag is required and tells the server whether to return extra information such as IDs, cross references, compound properties, and so on for each compound. Without it we primarily get just the information regarding the compounds connections in the network.

In [12]:
url = BASE_URL + f'get-comps/{DB_NAME}'
data = {'id_list': ['Ce0754b19f00e428c8f7762151e179ab9f468ad75',
                    'C32068d7a45048ac4d78c1eb182e46bd38db5cdad',
                    'Cae5a5e95558b05df93151078599e4f6d38ce0eff'],
        'return_extra_info': False}
response = requests.post(url, json=data, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Expand': True,
  'Generation': 0,
  'ID': 'C17116',
  'InChI_key': 'ZZGANZXITREHOP-UHFFFAOYSA-N',
  'Product_of': ['Ce0754b19f00e428c8f7762151e179ab9f468ad75_0'],
  'Reactant_in': ['Ce0754b19f00e428c8f7762151e179ab9f468ad75_0'],
  'SMILES': 'C#CC#CCc1ccccc1',
  'Type': 'Starting Compound',
  '_id': 'Ce0754b19f00e428c8f7762151e179ab9f468ad75'},
 {'Expand': True,
  'Generation': 0,
  'ID': 'C00762',
  'InChI_key': 'UORVCLMRJXCDCP-UHFFFAOYSA-N',
  'Product_of': ['C32068d7a45048ac4d78c1eb182e46bd38db5cdad_0'],
  'Reactant_in': ['C32068d7a45048ac4d78c1eb182e46bd38db5cdad_0'],
  'SMILES': 'C#CC(=O)O',
  'Type': 'Starting Compound',
  '_id': 'C32068d7a45048ac4d78c1eb182e46bd38db5cdad'},
 {'Expand': True,
  'Generation': 0,
  'ID': 'C14040',
  'InChI_key': 'PHNUZKMIPFFYSO-UHFFFAOYSA-N',
  'Product_of': [],
  'Reactant_in': ['Cae5a5e95558b05df93151078599e4f6d38ce0eff_0'],
  'SMILES': 'C#CC(C)(C)NC(=O)c1cc(Cl)cc(Cl)c1',
  'Type': 'Starting Compound',
  '_id': 'Cae5a5e95558b05df93151078599e4f6

### Example 2c - Get Reactions
Here, we get info for the first two reactions from get-ids for the reaction collection.

In [13]:
url = BASE_URL + f'get-rxns/{DB_NAME}'
data = {'id_list': ['R664033f4e61722ca696ae9be7c0d1e8fcbd63afa50350d121f40d461c560866a',
                    'R839b85080833082696be46bf63e4afbc54f4b4da6bc0812f2ff390967b69ec96']}
response = requests.post(url, json=data, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Operators': ['rule0023'],
  'Products': [[1,
    'C1cd07c7aabc87511a7b8412f300b5fa8538c108a',
    'O=C(O)C(Br)(Br)Br']],
  'Reactants': [[1, 'C0dbe6886e9129c6112d0d21fd718d86d5d3dc575', 'BrC(Br)Br'],
   [1, 'Xad024c97bc6d1c83baf5d8902af02c3a3db5358e', 'O=C=O']],
  'SMILES_rxn': '(1) BrC(Br)Br + (1) O=C=O => (1) O=C(O)C(Br)(Br)Br',
  '_id': 'R664033f4e61722ca696ae9be7c0d1e8fcbd63afa50350d121f40d461c560866a'},
 {'Operators': ['rule0110'],
  'Products': [[1,
    'Cd65ca39a895c400a350a8a53afe42e42235ca2fd',
    'CC(C)=CCC(Br)(Br)Br'],
   [1, 'Xa2a868d3f3866d6bfbbe78b615e42f81511ebff8', 'O=P(O)(O)OP(=O)(O)O']],
  'Reactants': [[1, 'C0dbe6886e9129c6112d0d21fd718d86d5d3dc575', 'BrC(Br)Br'],
   [1,
    'Xf4a7855630639e93324efbcfa9adb1b6a80e8b62',
    'CC(C)=CCOP(=O)(O)OP(=O)(O)O']],
  'SMILES_rxn': '(1) BrC(Br)Br + (1) CC(C)=CCOP(=O)(O)OP(=O)(O)O => (1) CC(C)=CCC(Br)(Br)Br + (1) O=P(O)(O)OP(=O)(O)O',
  '_id': 'R839b85080833082696be46bf63e4afbc54f4b4da6bc0812f2ff390967b69ec96'}]

### Example 2d - Get Operators
Here, we get info for two operators from get-ids for the operator colleciton.

In [14]:
url = BASE_URL + f'get-ops/{DB_NAME}'
data = {'id_list': ['rule0232', 'rule1231']}
response = requests.post(url, json=data, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'': '231',
  'Comments': 'RXN-14655;RXN-14656;RXN-14820;RXN-14821;RXN-14822;RXN-14823;RXN-8149;RXN-8152;RXN-9863;RXN-9864;RXN-9901',
  'Name': 'rule0232',
  'Products': ['Any', 'Any', 'O2'],
  'Reactants': ['Any', 'CO2', 'Any', 'Any'],
  'Reactions_predicted': 65684,
  'SMARTS': '[#6:1]-[#8:2].[#6:3]=[#8:4].[#6:5]=[#8:6].[#8:7]>>[#6:1]-[#6:3]-[#8:4].[#6:5]-[#8:7].[#8:2]=[#8:6]',
  '_id': 'rule0232'},
 {'': '1069',
  'Comments': 'HEPARITIN-SULFOTRANSFERASE-RXN',
  'Name': 'rule1231',
  'Products': ['Any', 'SULFATE_DONOR_CoF'],
  'Reactants': ['Any', 'SULFATE_ACCEPTOR_CoF'],
  'Reactions_predicted': 13,
  'SMARTS': '[#7:1]-[#16:2].[#8:3]>>[#7:1].[#8:3]-[#16:2]',
  '_id': 'rule1231'}]

### Example 2e - Get Reactions for Operator
Here, we get all reactions linked to a specific operator. Similar to get-ops but in this case we also get the reaction IDs.

In [15]:
url = BASE_URL + f'get-op-w-rxns/{DB_NAME}/rule1231'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'': '1069',
 'Comments': 'HEPARITIN-SULFOTRANSFERASE-RXN',
 'Name': 'rule1231',
 'Products': ['Any', 'SULFATE_DONOR_CoF'],
 'Reactants': ['Any', 'SULFATE_ACCEPTOR_CoF'],
 'Reaction_ids': ['R700bfbc7946a9c659017d165d9a56ec61fc76241edb34099a713a30b10fa350b',
  'R7f52b35d9f88ca52023ae28bb5e32dfe757b24da1858dfe59bdd5f49508050be',
  'Rdafb78b40e6fa5387c3aff657665ec04da06f84b29f5184f3d60d9c538422243',
  'R2744457f2eb7688d77d59c4db94a4ae1bc9e3f5ca62233bd4b122bf8aaa3e76b',
  'Rb6e1bf1b356a73741a16e6c0e5e0c7506bada8ee859f3452b2e2530009198154',
  'R9c4e02c892ed9fdf373b41f69095af71ad50dc81d8a167d6eec0b1976b71980e',
  'R750e9e716a7738269c508d598b961d8944d44c3b456912c6227e65ed947146ea',
  'Rb54cff849c2841233c9750f31cc6a4041393e2310b430f5a11cd546135a78715',
  'R489def7814720d729974c16413fec0acf73ed10cb1c06a513dfa5f24d976f164',
  'Red325c40798bf8fc8ff26a44fb4de6da81155e4ed74c98cd072d9f62a4275067',
  'Rcb21cd0b581d098cc61914d9c24ecc24b57ff9e0800138350bdbb84f64297d6e',
  'R71fa464385986822c1c43ea601e3

## 3. Perform an MS Search
Provide a monoisotopic mass as well as a dictionary of search parameters to search the MINE for matching compounds. This is a POST request. Here we just provide monoisotopic mass as regular text. However, you can also copy and paste mgf- or mzxml-formatted data in the 'text' field (and set 'text_type' to the appropriate value).

**Endpoint**: .../mineserver/ms-adduct-search/&lt;db_name>

**URL Arguments**:
- &lt;db_name>: name of MINE database

**JSON Arguments**:
- See API documentation.

### Example 3a - MS Search

In [16]:
url = BASE_URL + f'ms-adduct-search/{DB_NAME}'
json_dict = {
    'tolerance': 3,  # required (mDa)
    'charge': False,  # True=positive
    'text': '259.02244262600003',  # monoisotopic mass (including adduct) goes here
    # rest of parameters are optional
    'text_type': 'form',
    'adducts': "['[M-H]-']",
    'models': "['eco']",  # Filter to only comps that could be produced from E coli (as defined by KEGG)
    'ppm': False,
    'halogens': True,
    'verbose': True
}
response = requests.post(url, json=json_dict, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Formula': 'C9H6F6O2',
  'Inchikey': 'XBQISEWBYMWLET-UHFFFAOYSA-N',
  'MINE_id': 4028537,
  'SMILES': 'Oc1ccc(C(O)(C(F)(F)F)C(F)(F)F)cc1',
  'Spectra': {'Negative': {'10V': [[259.01992, 100.0]],
    '20V': [[189.01689, 21.86], [259.01992, 100.0]],
    '40V': [[189.01689, 18.22], [259.01992, 100.0]]},
   'Positive': {'10V': [[261.03448, 100.0]],
    '20V': [[261.03448, 100.0]],
    '40V': [[51.02293, 11.58],
     [53.03858, 8.58],
     [193.00826, 11.58],
     [199.01768, 29.16],
     [205.00826, 7.67],
     [207.02391, 8.25],
     [213.03333, 12.13],
     [217.00826, 13.86],
     [219.02391, 100.0],
     [231.02391, 15.57],
     [233.03956, 9.09],
     [241.02825, 15.58],
     [261.03448, 10.09]]}},
  '_id': 'Cc3d88f2fb038c43aa2e84c9c3161124e9a30bed9',
  'adduct': '[M-H]-',
  'logP': 2.7044000000000006,
  'peak_name': '259.02244262600003'},
 {'Formula': 'C11H13ClO3S',
  'Inchikey': 'HMLSQZYMGOSQDN-UHFFFAOYSA-N',
  'MINE_id': 2811674,
  'SMILES': 'CCSC(=O)C(O)Oc1ccc(Cl)cc1C',
  'Spect

In [17]:
len(json.loads(response.content))

884

## 4. Perform an MS2 Search
Provide a list of MS2 spectra and intensities as well as a dictionary of search parameters to search the MINE for matching compounds. This is a POST request. Here we just provide monoisotopic mass as regular text. However, you can also copy and paste mgf- or mzxml-formatted data in the 'text' field (and set 'text_type' to the appropriate value). Spectral score indicates the quality of the match (from 0 to 1000).

**Endpoint**: .../mineserver/ms2-search/&lt;db_name>

**URL Arguments**:
- &lt;db_name>: name of MINE database

**JSON Arguments**:
- See API documentation.

### Example 4a - MS2 Search

In [18]:
url = BASE_URL + f'ms2-search/{DB_NAME}'
json_dict = {
    'tolerance': 10,  # in mDa
    'charge': True,  # True=positive
    'energy_level': 20,  # in volts
    'scoring_function': 'dot product',
    'text': '261.037\n43.0189 1\n59.013 1\n96.970 10',  # MS2 spectra go here
    # All following parameters are optional
    'text_type': 'form',
    'adducts': "['[M+H]+']",
    #'models': "['hsa']",
    'ppm': False,
    'halogens': False,
    'verbose': True
}
response = requests.post(url, json=json_dict, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Formula': 'C6H13O9P',
  'Inchikey': 'OHHWZOLKNMHAAJ-UHFFFAOYSA-N',
  'MINE_id': 3594886,
  'SMILES': 'O=C(O)C(O)COP(=O)(O)OCC(O)CO',
  'Spectra': {'Negative': {'10V': [[78.95905, 100.0],
     [166.9751, 20.95],
     [171.0064, 50.84],
     [184.98566, 13.86],
     [259.02244, 73.69]],
    '20V': [[78.95905, 100.0],
     [96.96962, 28.29],
     [138.98018, 5.63],
     [152.99583, 13.48],
     [166.9751, 8.32],
     [168.99075, 13.71],
     [171.0064, 8.3]],
    '40V': [[78.95905, 100.0], [152.99583, 14.77]]},
   'Positive': {'10V': [[89.02332, 16.58],
     [96.96852, 16.73],
     [98.98417, 48.27],
     [140.99474, 12.78],
     [143.01039, 7.96],
     [155.01039, 13.89],
     [168.98965, 23.63],
     [173.02095, 11.37],
     [187.00022, 100.0]],
    '20V': [[89.02332, 11.08],
     [96.96852, 15.54],
     [98.98417, 100.0],
     [140.99474, 7.17],
     [168.98965, 9.35],
     [187.00022, 8.74]],
    '40V': [[75.04406, 6.81],
     [80.97361, 6.8],
     [82.98926, 10.43],
     [96.96852

In [19]:
len(json.loads(response.content))

1156

## 5. Download Predicted Spectra
In the MINE database, spectra are predicted for every compound using CFM-ID 4.0. You can retrieve these predictions by using this endpoint.

**Endpoint**: .../mineserver/spectra-download/&lt;mongo_id>

**URL Arguments**:
- &lt;mongo_id>: Mongo ID of compound to download spectra for 

### Example 5 - Download a Spectrum for a Specific Compound

In [20]:
url = BASE_URL + f'spectra-download/C0000256c516bdb0a11d68b1ecc2b066bad33ed64'
response = requests.get(url, verify=False)
response.content.decode().split('\n')

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['Name: MINE Compound C0000256c516bdb0a11d68b1ecc2b066bad33ed64',
 'Formula: C26H42O2',
 'Inchikey: YLOFSVMLRHSDAB-PZKBCHNNSA-N',
 'Mass: 386.31848058399993',
 'SMILES: CCC1CC1/C=C\\C/C=C\\C/C=C\\C/C=C\\CCCCCCCCCC(=O)O',
 'logP: 8.023100000000007',
 'Charge: 0',
 'MINE_id: 1489368',
 'KEGG_id: None',
 'Instrument: CFM-ID 4.0',
 'Ionization: Positive',
 'Energy: 10V',
 'Num Peaks: 30',
 '55.05423 15.75',
 '69.06988 16.26',
 '71.04914 10.44',
 '79.05423 6.74',
 '81.06988 8.87',
 '85.06479 5.28',
 '89.05971 5.32',
 '93.06988 8.0',
 '103.05423 5.48',
 '107.08553 5.15',
 '135.11683 6.58',
 '137.13248 8.58',
 '171.11683 11.96',
 '185.13248 10.42',
 '199.14813 5.88',
 '201.16378 8.44',
 '223.16926 5.48',
 '251.20056 6.43',
 '255.21073 5.43',
 '257.22638 11.01',
 '271.24203 10.13',
 '295.24203 6.5',
 '305.24751 5.27',
 '307.26316 6.62',
 '323.27333 12.42',
 '327.30463 8.58',
 '341.32028 7.3',
 '351.30463 69.29',
 '369.31519 85.68',
 '387.32576 100.0',
 '',
 'Name: MINE Compound C0000256c516bdb

## 6. Get List of Possible Metabolomics Adducts
Get names of adducts able to be used for metabolomics (MS and MS2) searches. This is a GET request.

**Endpoint**: .../mineserver/get-adduct-names/&lt;adduct_type>
where &lt;adduct_type> is optional

**URL Arguments**:
- &lt;adduct_type>: Type of adduct. Options are 'positive', 'negative', or 'all'. Defaults to 'all'.

### Example 6a - Get all Adduct Names

In [21]:
url = BASE_URL + 'get-adduct-names'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


[['[M]+',
  '[M+H]+',
  '[M+NH4]+',
  '[M+Na]+',
  '[M+2H]+',
  '[M+K]+',
  '[M+3H]+',
  '[M+2H+Na]+',
  '[M+H+2Na]+',
  '[M+3Na]+',
  '[M+H+NH4]+',
  '[M+H+Na]+',
  '[M+H+K]+',
  '[M+ACN+2H]+',
  '[M+2Na]+',
  '[M+2ACN+2H]+',
  '[M+3ACN+2H]+',
  '[M+CH3OH+H]+',
  '[M+ACN+H]+',
  '[M+2Na-H]+',
  '[M+IsoProp+H]+',
  '[M+ACN+Na]+',
  '[M+2K+H]+',
  '[M+DMSO+H]+',
  '[M+2ACN+H]+',
  '[M+IsoProp+Na+H]+',
  '[2M+H]+',
  '[2M+NH4]+',
  '[2M+Na]+',
  '[2M+3H2O+2H]+',
  '[2M+K]+',
  '[2M+ACN+H]+',
  '[2M+ACN+Na]+'],
 ['[M-H]-',
  '[M+CH3COO]-',
  '[M-2H]-',
  '[2M-H]-',
  '[M+H2O-2H]-',
  '[M+COO]-',
  '[M-3H]-',
  '[M+Na-2H]-',
  '[M+Cl]-',
  '[M+K-2H]-',
  '[M+Br]-',
  '[M+CF3COO]-',
  '[2M+COO]-',
  '[2M+CH3COO]-',
  '[3M-H]-',
  '[M+F]-']]

### Example 6b - Get Positive Adduct Names

In [22]:
url = BASE_URL + 'get-adduct-names/positive'
response = requests.get(url, verify=False)
json.loads(response.content)

/home/jonstrutz/anaconda3/envs/mine3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'minedatabase.ci.northwestern.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['[M]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+Na]+',
 '[M+2H]+',
 '[M+K]+',
 '[M+3H]+',
 '[M+2H+Na]+',
 '[M+H+2Na]+',
 '[M+3Na]+',
 '[M+H+NH4]+',
 '[M+H+Na]+',
 '[M+H+K]+',
 '[M+ACN+2H]+',
 '[M+2Na]+',
 '[M+2ACN+2H]+',
 '[M+3ACN+2H]+',
 '[M+CH3OH+H]+',
 '[M+ACN+H]+',
 '[M+2Na-H]+',
 '[M+IsoProp+H]+',
 '[M+ACN+Na]+',
 '[M+2K+H]+',
 '[M+DMSO+H]+',
 '[M+2ACN+H]+',
 '[M+IsoProp+Na+H]+',
 '[2M+H]+',
 '[2M+NH4]+',
 '[2M+Na]+',
 '[2M+3H2O+2H]+',
 '[2M+K]+',
 '[2M+ACN+H]+',
 '[2M+ACN+Na]+']